In [1]:
import torchvision.models as models
import torch
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

from tqdm import tqdm
import os
import datetime

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
path = os.getcwd()
path_train = os.getcwd() + '\\train'
path_val = os.getcwd() + '\\valid'
path_test = os.getcwd() + '\\test'

In [4]:
preprocess = transforms.Compose([
    transforms.Resize((112,112)),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.4723, 0.4682, 0.3930],
    #                      std=[0.2321, 0.2257, 0.2483])
])

In [5]:
train_set = ImageFolder(path_train, transform=preprocess)
train_set.class_to_idx
val_set = ImageFolder(path_val, transform=preprocess)
val_set.class_to_idx
test_set = ImageFolder(path_test, transform=preprocess)
test_set.class_to_idx

{'ABBOTTS BABBLER': 0,
 'ABBOTTS BOOBY': 1,
 'ABYSSINIAN GROUND HORNBILL': 2,
 'AFRICAN CROWNED CRANE': 3,
 'AFRICAN EMERALD CUCKOO': 4,
 'AFRICAN FIREFINCH': 5,
 'AFRICAN OYSTER CATCHER': 6,
 'AFRICAN PIED HORNBILL': 7,
 'ALBATROSS': 8,
 'ALBERTS TOWHEE': 9,
 'ALEXANDRINE PARAKEET': 10,
 'ALPINE CHOUGH': 11,
 'ALTAMIRA YELLOWTHROAT': 12,
 'AMERICAN AVOCET': 13,
 'AMERICAN BITTERN': 14,
 'AMERICAN COOT': 15,
 'AMERICAN FLAMINGO': 16,
 'AMERICAN GOLDFINCH': 17,
 'AMERICAN KESTREL': 18,
 'AMERICAN PIPIT': 19,
 'AMERICAN REDSTART': 20,
 'AMERICAN WIGEON': 21,
 'AMETHYST WOODSTAR': 22,
 'ANDEAN GOOSE': 23,
 'ANDEAN LAPWING': 24,
 'ANDEAN SISKIN': 25,
 'ANHINGA': 26,
 'ANIANIAU': 27,
 'ANNAS HUMMINGBIRD': 28,
 'ANTBIRD': 29,
 'ANTILLEAN EUPHONIA': 30,
 'APAPANE': 31,
 'APOSTLEBIRD': 32,
 'ARARIPE MANAKIN': 33,
 'ASHY STORM PETREL': 34,
 'ASHY THRUSHBIRD': 35,
 'ASIAN CRESTED IBIS': 36,
 'ASIAN DOLLARD BIRD': 37,
 'AUCKLAND SHAQ': 38,
 'AUSTRAL CANASTERO': 39,
 'AUSTRALASIAN FIGBIRD': 40,
 '

ResNet 18

In [6]:
def test_loop(model, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, predicted = torch.max(outputs, dim=1)
            total += labels.shape[0]
            correct += int((predicted == labels).sum())
    print("Model: {}, Accuracy: {}".format(model.name,
                                           correct / total))

In [7]:
def validation_loop(model, validation_loader, loss_fn):
    loss_val, correct, total = 0.0, 0.0, 0.0
    with torch.no_grad():
        for imgs, labels in validation_loader:
            imgs = imgs.to(device=device); labels = labels.to(device=device)
            outputs = model(imgs)
            val_loss = loss_fn(outputs,labels)
            loss_val += val_loss.item()
            outputs = model(imgs)
            _, predicted = torch.max(outputs, dim=1)
            total += labels.shape[0]
            correct += int((predicted == labels).sum())

    return loss_val, correct/total

In [8]:
def training_loop(model, optimizer, loss_fn, train_loader):
    loss_train, correct, total = 0.0, 0.0, 0.0
    for imgs, labels in tqdm(train_loader):
        imgs = imgs.to(device=device); labels = labels.to(device=device)
        outputs = model(imgs)
        loss = loss_fn(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_train += loss.item()
        outputs = model(imgs)
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())

    
    return loss_train, correct/total

In [9]:
def training(n_epochs, optimizer, model, loss_fn, train_loader, validation_loader):

    past_loss = float('inf')
    best_epoch = 1

    for epoch in range(1, n_epochs + 1):

        # stop training if we detect overfitting
        if epoch > best_epoch + 3:
            break

        # training 
        loss_train, accuracy_train = training_loop(model, optimizer, loss_fn, train_loader)

        # validation
        loss_val, accuracy_val = validation_loop(model, validation_loader, loss_fn)

        # saving best model state
        if (loss_val < past_loss):
            print('Saving Model...')
            torch.save(model.state_dict(), path + '\\{}_best_model_state.pth'.format(model.name))
            past_loss = loss_val
            best_epoch = epoch
                
        print('{} Epoch {}, Training loss {}, Accuracy {}'.format(datetime.datetime.now(), epoch,
                                                                  loss_train / len(train_loader),
                                                                  accuracy_train))

        print('{} Epoch {}, Validation loss {}, Accuracy {}'.format(datetime.datetime.now(), epoch,
                                                                    loss_val / len(validation_loader),
                                                                    accuracy_val))

In [10]:
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
val_loader = DataLoader(val_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

In [11]:
ResNet18 = models.resnet18(weights=None); ResNet18.name = 'ResNet18'
VGG11 = models.vgg11_bn(weights=None); VGG11.name = 'VGG11'
VGG19 = models.vgg19_bn(weights=None); VGG19.name = 'VGG19'
EfficientNetB0 = models.efficientnet_b0(weights=None); EfficientNetB0.name = 'EfficientNetB0'
DenseNet121 = models.densenet121(weights=None); DenseNet121.name = 'DenseNet121'

models_list = [ResNet18, VGG11, VGG19, EfficientNetB0, DenseNet121]

In [12]:
n_epochs = 35
loss_fn = nn.CrossEntropyLoss()

In [ ]:
for model in models_list:    
    print(model.name)
    optimizer = optim.SGD(model.parameters(), lr=1e-2)
    training(
        n_epochs = n_epochs,
        optimizer = optimizer,
        model = model.to(device=device),
        loss_fn = loss_fn,
        train_loader = train_loader,
        validation_loader = val_loader
    )

In [16]:
ResNet18.load_state_dict(torch.load(path + '\\ResNet18_best_model_state.pth'))
ResNet18.eval()
VGG11.load_state_dict(torch.load(path + '\\VGG11_best_model_state.pth'))
VGG11.eval()
VGG19.load_state_dict(torch.load(path + '\\VGG19_best_model_state.pth'))
VGG19.eval()
EfficientNetB0.load_state_dict(torch.load(path + '\\EfficientNetB0_best_model_state.pth'))
EfficientNetB0.eval()
DenseNet121.load_state_dict(torch.load(path + '\\DenseNet121_best_model_state.pth'))
DenseNet121.eval()

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Python\\Bird_classificationResNet18_best_model_state.pth'

In [15]:
models_list = [ResNet18, VGG11, VGG19, EfficientNetB0, DenseNet121]

In [14]:
for model in models_list:
    test_loop(model.to(device=device), test_loader)

Model: EfficientNetB0, Accuracy: 0.0017777777777777779
Model: DenseNet121, Accuracy: 0.0022222222222222222
